In [39]:
# Loading the data

In [40]:
import pandas as pd
df_graph = pd.read_csv('/ivi/ilps/personal/vprovat/valkema_graph_directed_manual_FINAL.tsv',sep='\t')

In [41]:
df_graph.head()

,person1,person2,document_ids
0,J. (Han) Feenstra,Sybren Valkema,['1310445']
1,Jan Antonie de Melker,Sybren Valkema,"['1262435', '1262439', '1262454']"
2,A. Binter,Sybren Valkema,"['1269270', '1269271', '1269272']"
3,A. Binter,Veronique Valkema,"['1269270', '1269271', '1269272']"
4,A. Jansen,Sybren Valkema,"['1459844', '1459845', '1459846', '1459847']"


In [42]:
df_graph['weight'] = df_graph['document_ids'].map(lambda x: len(eval(x)))

In [43]:
df_graph.nunique()

person1         195
person2          36
document_ids    230
weight           23
dtype: int64

In [56]:
df_graph

,person1,person2,document_ids,weight
0,J. (Han) Feenstra,Sybren Valkema,['1310445'],1
1,Jan Antonie de Melker,Sybren Valkema,"['1262435', '1262439', '1262454']",3
2,A. Binter,Sybren Valkema,"['1269270', '1269271', '1269272']",3
3,A. Binter,Veronique Valkema,"['1269270', '1269271', '1269272']",3
4,A. Jansen,Sybren Valkema,"['1459844', '1459845', '1459846', '1459847']",4
...,...,...,...,...
254,Sybren Valkema,sochar Rajko Georgiev Rajkov,"['1453564', '1453571', '1453572', '1453573', '...",9
255,Sybren Valkema,students,"['1269476', '1269477', '1269478']",3
256,Sybren Valkema,wy herman hart,['1297980'],1
257,Veronique Valkema,Zaharah Hughes,['1288898'],1


In [45]:
import pickle
country_by_person = pickle.load(open('country_per_person.p','rb'))

In [46]:
colours = {'United States of America':'#1E90FF',#blue
           'Kingdom of the Netherlands': '#FFA500',#orange
          }

def get_colour(person):
    country = country_by_person[person] if person in country_by_person else ''
    if country in colours:
        return colours[country]
    return '#9370DB' # purple


In [47]:
import networkx as nx


In [48]:
G = nx.Graph()
for _,row in df_graph.iterrows():
    if row['weight'] > 1: # pruning smallest connections
        G.add_edge(row['person1'],row['person2'],weight=row['weight'])

In [54]:
from pyvis.network import Network
import numpy as np
def prepare_network(G1):
    # Remove components that are too small (for visualisation purposes)
    for component in list(nx.connected_components(G1)):
        if len(component)<3:
            for node in component:
                G1.remove_node(node)
    # Calculate node degrees
    node_degrees = dict(G1.degree())
    
    net = Network(notebook=True,cdn_resources='remote',bgcolor='#f5f5f5',height='600px',width='600px')
    
    # Define a scaling factor for font size (adjust as needed)
    font_scale = 15
    node_scale = 10#0

    # Add nodes with sizes and font sizes proportional to their degrees
    for node_id, degree in node_degrees.items():
        node_size = np.log(degree*node_scale)#np.log2(degree) * node_scale # Scale node size (adjust as needed)
        font_size = min(2*np.sqrt(degree*font_scale),
                        20)
                        #degree*font_scale#np.log(degree)*font_scale# * font_scale  # Scale font size
        colour = get_colour(node_id)
        net.add_node(node_id, size=node_size, label=str(node_id),color=colour,
                     shape='dot',
                     font={'size': font_size,'color': 'black'},#colour},
                    labelHighlightBold=True)

    # Add edges with weights
    for u, v, data in G.edges(data=True):
        weight = data['weight']  # Extract the weight of the edge
        net.add_edge(u, v, value=weight,
                     color='#d6ebc3')#average_hex_colors(get_colour(u),get_colour(v)))  # Use 'value' to set the edge weight in Pyvis
        
    # Configure physics for repulsion
        
    net.repulsion(
        node_distance=200,    # Distance between nodes
        central_gravity=7.55, # Strength of the central gravity
        spring_length=100,    # Length of the springs
        spring_strength=0.05, # Strength of the springs
        damping=0.6#,          # Damping factor to stabilize the layout
#         avoid_overlap=1
    )

    net.toggle_physics(True)
    
    return net

In [55]:
net = prepare_network(G)
net.show_buttons(filter_=["physics"])
net.show('graph-manual-final.html')
# net.write_graphml('valkema.graphml')


graph-manual-final.html
